In [35]:
import numpy as np
import tqdm
from IPython.display import HTML, display

In [36]:
teams = {
    "Real Madrid": {"nation": "Spain", "coefficient": 136.000, "pot": 1},
    "Manchester City": {"nation": "England", "coefficient": 148.000, "pot": 1},
    "Bayern Munich": {"nation": "Germany", "coefficient": 144.000, "pot": 1},
    "Paris Saint-Germain": {"nation": "France", "coefficient": 116.000, "pot": 1},
    "Liverpool": {"nation": "England", "coefficient": 114.000, "pot": 1},
    "Inter": {"nation": "Italy", "coefficient": 101.000, "pot": 1},
    "Borussia Dortmund": {"nation": "Germany", "coefficient": 97.000, "pot": 1},
    "RB Leipzig": {"nation": "Germany", "coefficient": 97.000, "pot": 1},
    "Barcelona": {"nation": "Spain", "coefficient": 91.000, "pot": 1},
    "Bayer Leverkusen": {"nation": "Germany", "coefficient": 90.000, "pot": 2},
    "Atlético Madrid": {"nation": "Spain", "coefficient": 89.000, "pot": 2},
    "Atalanta": {"nation": "Italy", "coefficient": 81.000, "pot": 2},
    "Juventus": {"nation": "Italy", "coefficient": 80.000, "pot": 2},
    "Benfica": {"nation": "Portugal", "coefficient": 79.000, "pot": 2},
    "Arsenal": {"nation": "England", "coefficient": 72.000, "pot": 2},
    "Club Brugge": {"nation": "Belgium", "coefficient": 64.000, "pot": 2},
    "Shakhtar Donetsk": {"nation": "Ukraine", "coefficient": 63.000, "pot": 2},
    "Milan": {"nation": "Italy", "coefficient": 59.000, "pot": 2},
    "Feyenoord": {"nation": "Netherlands", "coefficient": 57.000, "pot": 3},
    "Sporting CP": {"nation": "Portugal", "coefficient": 54.500, "pot": 3},
    "PSV Eindhoven": {"nation": "Netherlands", "coefficient": 54.000, "pot": 3},
    "Dinamo Zagreb": {"nation": "Croatia", "coefficient": 50.000, "pot": 3},
    "Red Bull Salzburg": {"nation": "Austria", "coefficient": 50.000, "pot": 3},
    "Lille": {"nation": "France", "coefficient": 47.000, "pot": 3},
    "Red Star Belgrade": {"nation": "Serbia", "coefficient": 40.000, "pot": 3},
    "Young Boys": {"nation": "Switzerland", "coefficient": 34.500, "pot": 3},
    "Celtic": {"nation": "Scotland", "coefficient": 32.000, "pot": 3},
    "Slovan Bratislava": {"nation": "Slovakia", "coefficient": 30.500, "pot": 4},
    "Monaco": {"nation": "France", "coefficient": 24.000, "pot": 4},
    "Sparta Prague": {"nation": "Czech Republic", "coefficient": 22.500, "pot": 4},
    "Aston Villa": {"nation": "England", "coefficient": 20.860, "pot": 4},
    "Bologna": {"nation": "Italy", "coefficient": 18.056, "pot": 4},
    "Girona": {"nation": "Spain", "coefficient": 17.897, "pot": 4},
    "VfB Stuttgart": {"nation": "Germany", "coefficient": 17.324, "pot": 4},
    "Sturm Graz": {"nation": "Austria", "coefficient": 14.500, "pot": 4},
    "Brest": {"nation": "France", "coefficient": 13.366, "pot": 4},
}


In [37]:
matches = [
    ("Real Madrid", "Borussia Dortmund"),
    ("Real Madrid", "Milan"),
    ("Real Madrid", "Red Bull Salzburg"),
    ("Real Madrid", "VfB Stuttgart"),
    ("Manchester City", "Inter"),
    ("Manchester City", "Club Brugge"),
    ("Manchester City", "Feyenoord"),
    ("Manchester City", "Sparta Prague"),
    ("Bayern Munich", "Paris Saint-Germain"),
    ("Bayern Munich", "Benfica"),
    ("Bayern Munich", "Dinamo Zagreb"),
    ("Bayern Munich", "Slovan Bratislava"),
    ("Paris Saint-Germain", "Manchester City"),
    ("Paris Saint-Germain", "Atlético Madrid"),
    ("Paris Saint-Germain", "PSV Eindhoven"),
    ("Paris Saint-Germain", "Girona"),
    ("Liverpool", "Real Madrid"),
    ("Liverpool", "Bayer Leverkusen"),
    ("Liverpool", "Lille"),
    ("Liverpool", "Bologna"),
    ("Inter", "RB Leipzig"),
    ("Inter", "Arsenal"),
    ("Inter", "Red Star Belgrade"),
    ("Inter", "Monaco"),
    ("Borussia Dortmund", "Barcelona"),
    ("Borussia Dortmund", "Shakhtar Donetsk"),
    ("Borussia Dortmund", "Celtic"),
    ("Borussia Dortmund", "Sturm Graz"),
    ("RB Leipzig", "Liverpool"),
    ("RB Leipzig", "Juventus"),
    ("RB Leipzig", "Sporting CP"),
    ("RB Leipzig", "Aston Villa"),
    ("Barcelona", "Bayern Munich"),
    ("Barcelona", "Atalanta"),
    ("Barcelona", "Young Boys"),
    ("Barcelona", "Brest"),
    ("Bayer Leverkusen", "Inter"),
    ("Bayer Leverkusen", "Milan"),
    ("Bayer Leverkusen", "Red Bull Salzburg"),
    ("Bayer Leverkusen", "Sparta Prague"),
    ("Atlético Madrid", "RB Leipzig"),
    ("Atlético Madrid", "Bayer Leverkusen"),
    ("Atlético Madrid", "Lille"),
    ("Atlético Madrid", "Slovan Bratislava"),
    ("Atalanta", "Real Madrid"),
    ("Atalanta", "Arsenal"),
    ("Atalanta", "Celtic"),
    ("Atalanta", "Sturm Graz"),
    ("Juventus", "Manchester City"),
    ("Juventus", "Benfica"),
    ("Juventus", "PSV Eindhoven"),
    ("Juventus", "VfB Stuttgart"),
    ("Benfica", "Barcelona"),
    ("Benfica", "Atlético Madrid"),
    ("Benfica", "Feyenoord"),
    ("Benfica", "Bologna"),
    ("Arsenal", "Paris Saint-Germain"),
    ("Arsenal", "Shakhtar Donetsk"),
    ("Arsenal", "Dinamo Zagreb"),
    ("Arsenal", "Monaco"),
    ("Club Brugge", "Borussia Dortmund"),
    ("Club Brugge", "Juventus"),
    ("Club Brugge", "Sporting CP"),
    ("Club Brugge", "Aston Villa"),
    ("Shakhtar Donetsk", "Bayern Munich"),
    ("Shakhtar Donetsk", "Atalanta"),
    ("Shakhtar Donetsk", "Young Boys"),
    ("Shakhtar Donetsk", "Brest"),
    ("Milan", "Liverpool"),
    ("Milan", "Club Brugge"),
    ("Milan", "Red Star Belgrade"),
    ("Milan", "Girona"),
    ("Feyenoord", "Bayern Munich"),
    ("Feyenoord", "Bayer Leverkusen"),
    ("Feyenoord", "Red Bull Salzburg"),
    ("Feyenoord", "Sparta Prague"),
    ("Sporting CP", "Manchester City"),
    ("Sporting CP", "Arsenal"),
    ("Sporting CP", "Lille"),
    ("Sporting CP", "Bologna"),
    ("PSV Eindhoven", "Liverpool"),
    ("PSV Eindhoven", "Shakhtar Donetsk"),
    ("PSV Eindhoven", "Sporting CP"),
    ("PSV Eindhoven", "Girona"),
    ("Dinamo Zagreb", "Borussia Dortmund"),
    ("Dinamo Zagreb", "Milan"),
    ("Dinamo Zagreb", "Celtic"),
    ("Dinamo Zagreb", "Monaco"),
    ("Red Bull Salzburg", "Paris Saint-Germain"),
    ("Red Bull Salzburg", "Atlético Madrid"),
    ("Red Bull Salzburg", "Dinamo Zagreb"),
    ("Red Bull Salzburg", "Brest"),
    ("Lille", "Real Madrid"),
    ("Lille", "Juventus"),
    ("Lille", "Feyenoord"),
    ("Lille", "Sturm Graz"),
    ("Red Star Belgrade", "Barcelona"),
    ("Red Star Belgrade", "Benfica"),
    ("Red Star Belgrade", "PSV Eindhoven"),
    ("Red Star Belgrade", "VfB Stuttgart"),
    ("Young Boys", "Inter"),
    ("Young Boys", "Atalanta"),
    ("Young Boys", "Red Star Belgrade"),
    ("Young Boys", "Aston Villa"),
    ("Celtic", "RB Leipzig"),
    ("Celtic", "Club Brugge"),
    ("Celtic", "Young Boys"),
    ("Celtic", "Slovan Bratislava"),
    ("Slovan Bratislava", "Manchester City"),
    ("Slovan Bratislava", "Milan"),
    ("Slovan Bratislava", "Dinamo Zagreb"),
    ("Slovan Bratislava", "VfB Stuttgart"),
    ("Monaco", "Barcelona"),
    ("Monaco", "Benfica"),
    ("Monaco", "Red Star Belgrade"),
    ("Monaco", "Aston Villa"),
    ("Sparta Prague", "Inter"),
    ("Sparta Prague", "Atlético Madrid"),
    ("Sparta Prague", "Red Bull Salzburg"),
    ("Sparta Prague", "Brest"),
    ("Aston Villa", "Bayern Munich"),
    ("Aston Villa", "Juventus"),
    ("Aston Villa", "Celtic"),
    ("Aston Villa", "Bologna"),
    ("Bologna", "Borussia Dortmund"),
    ("Bologna", "Shakhtar Donetsk"),
    ("Bologna", "Lille"),
    ("Bologna", "Monaco"),
    ("Girona", "Liverpool"),
    ("Girona", "Arsenal"),
    ("Girona", "Feyenoord"),
    ("Girona", "Slovan Bratislava"),
    ("VfB Stuttgart", "Paris Saint-Germain"),
    ("VfB Stuttgart", "Atalanta"),
    ("VfB Stuttgart", "Young Boys"),
    ("VfB Stuttgart", "Sparta Prague"),
    ("Sturm Graz", "RB Leipzig"),
    ("Sturm Graz", "Club Brugge"),
    ("Sturm Graz", "Sporting CP"),
    ("Sturm Graz", "Girona"),
    ("Brest", "Real Madrid"),
    ("Brest", "Bayer Leverkusen"),
    ("Brest", "PSV Eindhoven"),
    ("Brest", "Sturm Graz"),
]

print(f"Number of matches: {len(matches)}")

Number of matches: 144


In [38]:
def check_matches(matches):
    # count the matches for each team and the matches where a team is the first element
    team_matches = {}
    first_team_matches = {}
    for match in matches:
        for team in match:
            if team not in team_matches:
                team_matches[team] = 0
            team_matches[team] += 1
            if team == match[0]:
                if team not in first_team_matches:
                    first_team_matches[team] = 0
                first_team_matches[team] += 1
    return team_matches, first_team_matches


total_matches, first_team_matches = check_matches(matches)

print(total_matches)
print(first_team_matches)

{'Real Madrid': 8, 'Borussia Dortmund': 8, 'Milan': 8, 'Red Bull Salzburg': 8, 'VfB Stuttgart': 8, 'Manchester City': 8, 'Inter': 8, 'Club Brugge': 8, 'Feyenoord': 8, 'Sparta Prague': 8, 'Bayern Munich': 8, 'Paris Saint-Germain': 8, 'Benfica': 8, 'Dinamo Zagreb': 8, 'Slovan Bratislava': 8, 'Atlético Madrid': 8, 'PSV Eindhoven': 8, 'Girona': 8, 'Liverpool': 8, 'Bayer Leverkusen': 8, 'Lille': 8, 'Bologna': 8, 'RB Leipzig': 8, 'Arsenal': 8, 'Red Star Belgrade': 8, 'Monaco': 8, 'Barcelona': 8, 'Shakhtar Donetsk': 8, 'Celtic': 8, 'Sturm Graz': 8, 'Juventus': 8, 'Sporting CP': 8, 'Aston Villa': 8, 'Atalanta': 8, 'Young Boys': 8, 'Brest': 8}
{'Real Madrid': 4, 'Manchester City': 4, 'Bayern Munich': 4, 'Paris Saint-Germain': 4, 'Liverpool': 4, 'Inter': 4, 'Borussia Dortmund': 4, 'RB Leipzig': 4, 'Barcelona': 4, 'Bayer Leverkusen': 4, 'Atlético Madrid': 4, 'Atalanta': 4, 'Juventus': 4, 'Benfica': 4, 'Arsenal': 4, 'Club Brugge': 4, 'Shakhtar Donetsk': 4, 'Milan': 4, 'Feyenoord': 4, 'Sporting CP'

In [39]:
def coeff_based_prob(team1, team2):
    team1_coef = team1["coefficient"]
    team2_coef = team2["coefficient"]
    p = [
        team1_coef / (team1_coef + team2_coef),
        1 / (team1_coef + team2_coef),
        team2_coef / (team1_coef + team2_coef),
    ]
    p = [x / sum(p) for x in p]

    return p

In [49]:
def simulate_match(p=0.5):
    result = np.random.choice(["win", "draw", "lose"], p=p)

    if result == "win":
        return 3, 0
    elif result == "draw":
        return 1, 1
    else:
        return 0, 3

def simulate_matches(teams, matches, p_fn=coeff_based_prob):
    classification = {t: 0 for t in teams}

    for match in matches:
        
        p = p_fn(teams[match[0]], teams[match[1]])

        score1, score2 = simulate_match(p=p)
        classification[match[0]] += score1
        classification[match[1]] += score2
    return classification

def simulate(runs, teams, matches, p_fn=coeff_based_prob):
    aggregate_results = {t: {"avg": None, "top": None, "bottom": None} for t in teams}

    for i in tqdm.tqdm(range(runs)):
        results_dict = simulate_matches(teams, matches, )

        results = [(r, results_dict[r]) for r in results_dict]

        results.sort(key=lambda x: x[1], reverse=True)
        for i, (t, p) in enumerate(results):
            if aggregate_results[t]["avg"] is None:
                aggregate_results[t]["avg"] = p / runs
            else:
                aggregate_results[t]["avg"] += p / runs
            if aggregate_results[t]["top"] is None or i + 1 > aggregate_results[t]["top"]:
                aggregate_results[t]["top"] = i + 1
            if (
                aggregate_results[t]["bottom"] is None
                or i + 1 < aggregate_results[t]["bottom"]
            ):
                aggregate_results[t]["bottom"] = i + 1

    results = [
        (
            t,
            aggregate_results[t]["avg"],
            aggregate_results[t]["top"],
            aggregate_results[t]["bottom"],
        )
        for t in aggregate_results
    ]

    results.sort(key=lambda x: x[1], reverse=True)
    return results

In [51]:
def print_results(results):
    rows = []
    for i, (t, avg, worst, top) in enumerate(results):
        pos = i + 1
        row = [pos, t, round(avg), top, worst, "#000"]
        # print(pos, t, f"Media punti {round(avg)}", f"Miglior piazzamento {top}", f"Peggior piazzamento {bottom}")

        if pos >= 24:
            row[-1] = "#E3735E"
        elif i >= 8:
            row[-1] = "#FFBF00"
        rows.append(row)

    content = "".join(
        [f"<tr ><td style='color:{row[-1]}'>{row[0]}</td><td>{row[1]}</td><td>{row[2]}</td><td>{row[3]}</td><td>{row[4]}</td></tr>" for row in rows]
    )   

    display(
        HTML(
            f"<table><tr><th>Pos</th><th>Team</th><th>Points</th><th>Best</th><th>Worst</th></tr>{content}</table>"
        )
    )

In [52]:
runs = 10_000


results = simulate(runs, teams, matches, p_fn=coeff_based_prob)

100%|██████████| 10000/10000 [00:08<00:00, 1172.68it/s]


In [53]:
print_results(results)

Pos,Team,Points,Best,Worst
1,Real Madrid,17,1,33
2,Bayern Munich,17,1,36
3,Manchester City,17,1,36
4,Liverpool,16,1,35
5,Borussia Dortmund,16,1,36
6,Paris Saint-Germain,16,1,36
7,Inter,15,1,36
8,RB Leipzig,15,1,36
9,Atalanta,15,1,36
10,Barcelona,15,1,36
